In [1]:
#Sources:
#https://qmind.ca/static/media/QMIND-Project-Lookbook-2020-21.16f516dc.pdf 
# Converted first 5 projects to docx, and next 5 to PDFs; could have downloaded it as one data source

#https://2021.cucai.ca/CUCAI_2021_Proceedings.pdf (Can be used for later)

# Import dependencies
# Loading data
import os
import pandas as pd
import numpy as np

#import docx2text
#from pdf2text import *

In [3]:
# Text analysis
#import texthero as hero

In [4]:
from texthero import preprocessing

ModuleNotFoundError: No module named 'wordcloud.query_integral_image'

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
import tensorflow_hub as hub
# Download Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

2022-01-23 16:51:29.260939: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-23 16:51:29.263032: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# Empty list to store text data from files
import os
data = []

# Load each file in "./data" folder
for f in os.listdir('/Users/braulioantonio/Documents/Git/QMIND-knowledge-graph-project/Development/datasets/qmind_projects'):
    
    # If the file ends in .docx
    if f.endswith('.docx'):
        
        # Read in the text in the .docx file
        data.append(docx2text.process('./data/' + f))
    
    # Else if the file ends in .pdf
    elif f.endswith('.pdf'): 
        
        # Merge all the pages of text into one paragraph and add it to data list
        data.append(' '.join(str(word) for word in readPDF('./data/' + f).values()))
    
# Print each element in data
for i in data:
    print(i + "\n")

NameError: name 'docx2text' is not defined

In [16]:
# Extract title of project
title = [text.partition("\n")[0] for text in data]

# Delete title from each element in data
for i in range(len(data)):
    data[i] = data[i].replace(title[i], "")

# Create df
df = pd.DataFrame(list(zip(title, data)),columns =['projName', 'Desc'])

# Create text column
df["text"] = df["projName"] + " " + df["Desc"]

# Display
df

,projName,Desc,text
0,Ace Interview Prep: Evaluating Confidence for ...,"\n\nFor job seekers around the world, the recr...",Ace Interview Prep: Evaluating Confidence for ...
1,Forest Ecosystem Solutions: Forest Ecosystem A...,\nWith the growing occurrence of natural disas...,Forest Ecosystem Solutions: Forest Ecosystem A...
2,Kingston General Hospital: Forecasting Hospita...,\n\nCurrently in the Canadian health care syst...,Kingston General Hospital: Forecasting Hospita...
3,Loblaw Digital: Improving Product Categorizati...,\n\nTo organize their online product catalogue...,Loblaw Digital: Improving Product Categorizati...
4,LocateMotion: Diabetes Risk Prediction on Hete...,"\nIn 2020, an estimated 29% of Canadians are l...",LocateMotion: Diabetes Risk Prediction on Hete...
5,MeshAI: Hospital Scheduling,\nOur goal was to use a decision tree algorith...,MeshAI: Hospital Scheduling \nOur goal was to...
6,Loblaw Digital: Improving Product Categorizati...,\n\nTo organize their online product catalogue...,Loblaw Digital: Improving Product Categorizati...
7,Queen’s University PPS: Android Waste Classifi...,"\n\nMany people, especially Queen’s students u...",Queen’s University PPS: Android Waste Classifi...
8,RBC: Topic Modelling,"\n\nUnfortunately, specifics of the project ca...","RBC: Topic Modelling \n\nUnfortunately, specif..."
9,TD Bank: Investing,"\nUnfortunately, specifics of the project cann...","TD Bank: Investing \nUnfortunately, specifics..."


In [18]:
# Clean text
df['text'] = [n.replace('\n', '') for n in df['text']]
df['text'] = [n.replace('\t', '') for n in df['text']]
df['text'] = [n.replace('\r', '') for n in df['text']]

custom_pipeline = [preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_diacritics,
                   preprocessing.drop_no_content,
                   preprocessing.remove_punctuation,
                   preprocessing.remove_digits,
                   #preprocessing.stop_words,
                   preprocessing.remove_whitespace
                  ]

# Removing brackets using texthero removes contents within them, 
# so have to do those individually
df['clean_text'] = hero.clean(df['text'], custom_pipeline)
df['clean_text'] = [n.replace('{','') for n in df['clean_text']]
df['clean_text'] = [n.replace('}','') for n in df['clean_text']]
df['clean_text'] = [n.replace('(','') for n in df['clean_text']]
df['clean_text'] = [n.replace(')','') for n in df['clean_text']]

# Select columns
df = df[["projName", "clean_text"]]

# Display
df

,projName,clean_text
0,Ace Interview Prep: Evaluating Confidence for ...,ace interview prep evaluating confidence for i...
1,Forest Ecosystem Solutions: Forest Ecosystem A...,forest ecosystem solutions forest ecosystem an...
2,Kingston General Hospital: Forecasting Hospita...,kingston general hospital forecasting hospital...
3,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...
4,LocateMotion: Diabetes Risk Prediction on Hete...,locatemotion diabetes risk prediction on heter...
5,MeshAI: Hospital Scheduling,meshai hospital scheduling our goal was to use...
6,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...
7,Queen’s University PPS: Android Waste Classifi...,queen s university pps android waste classific...
8,RBC: Topic Modelling,rbc topic modelling unfortunately specifics of...
9,TD Bank: Investing,td bank investing unfortunately specifics of t...


In [24]:
# Doc2Vec
# Tokenize and tag the text
text_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df["clean_text"])]

# Display the tagged docs
text_docs

[TaggedDocument(words=['ace', 'interview', 'prep', 'evaluating', 'confidence', 'for', 'interview', 'preparation', 'for', 'job', 'seekers', 'around', 'the', 'world', 'the', 'recruiting', 'process', 'can', 'be', 'a', 'daunting', 'and', 'nerve', 'racking', 'endeavour', 'particularly', 'when', 'it', 'comes', 'time', 'for', 'an', 'interview', 'to', 'better', 'prepare', 'jobseekers', 'we', 'have', 'partnered', 'with', 'a', 'waterloo', 'based', 'start', 'up', 'that', 'provides', 'feedback', 'on', 'mock', 'interview', 'tailored', 'to', 'the', 'role', 's', 'they', 'are', 'applying', 'for', 'evaluating', 'an', 'interview', 'is', 'often', 'very', 'subjective', 'with', 'a', 'successful', 'interview', 'incorporating', 'a', 'wide', 'range', 'of', 'factors', 'including', 'accurate', 'content', 'engaging', 'delivery', 'all', 'supported', 'by', 'appropriate', 'non', 'verbal', 'communication', 'the', 'complexity', 'and', 'intricacy', 'of', 'an', 'interview', 'makes', 'it', 'difficult', 'to', 'provide', 

In [20]:
# Source for Hyperparameter tuning
# https://medium.com/betacom/hyperparameters-tuning-tf-idf-and-doc2vec-models-73dd418b4d

# Instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs=40)

# Build vocab
model.build_vocab(text_docs)

# Train model
model.train(text_docs, total_examples=model.corpus_count, epochs=model.epochs)

# Generate vectors
proj2vec = [model.infer_vector((df['clean_text'][i].split(' '))) for i in range(0,len(df['clean_text']))]

# Display
proj2vec

[array([ 0.6219966 , -0.5964304 , -1.9640135 ,  0.2550926 , -1.3556584 ,
         1.4725534 , -0.06052277,  0.77631694, -1.8542454 , -1.1576345 ,
        -1.8580128 , -0.12876521,  1.2487639 , -0.8415933 ,  2.506198  ,
         1.3066005 ,  0.28950045, -1.7200662 , -0.632111  , -0.27386448,
        -0.04748596, -0.9756765 , -0.10016135, -1.6808399 , -0.41863716,
         0.5479983 , -0.37997046, -0.9615907 ,  0.15547621,  1.3793939 ,
        -1.0549879 , -0.20669967,  2.2728212 , -0.14784747,  0.8869149 ,
        -0.26505733,  1.3550379 ,  1.4568068 ,  1.4832933 ,  0.39008883,
         1.5476558 ,  0.77802837,  2.47682   , -0.26365656,  1.651077  ,
        -1.2882019 ,  0.3770283 ,  0.17613684, -0.62996465, -0.24794298,
         0.81222856, -0.43863884, -0.29380548, -0.56452847,  0.45064607,
         0.4968208 ,  0.19630423,  0.5448014 ,  0.47451591,  1.7759663 ,
         3.588415  , -0.42000243,  0.8397946 , -0.04702903], dtype=float32),
 array([ 0.79333985, -0.5449119 , -1.9608767 , 

In [23]:
# Set list to dataframe column
df['proj2vec'] = np.array(proj2vec).tolist()

# Display
df

<ipython-input-23-e45abf4dfca6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['proj2vec'] = dtv


,projName,clean_text,proj2vec
0,Ace Interview Prep: Evaluating Confidence for ...,ace interview prep evaluating confidence for i...,"[0.6219965815544128, -0.5964304208755493, -1.9..."
1,Forest Ecosystem Solutions: Forest Ecosystem A...,forest ecosystem solutions forest ecosystem an...,"[0.7933398485183716, -0.5449119210243225, -1.9..."
2,Kingston General Hospital: Forecasting Hospita...,kingston general hospital forecasting hospital...,"[0.7627487182617188, -0.6631572246551514, -1.9..."
3,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...,"[0.9004322290420532, -0.3092217445373535, -2.0..."
4,LocateMotion: Diabetes Risk Prediction on Hete...,locatemotion diabetes risk prediction on heter...,"[0.9695004820823669, -0.7922118902206421, -1.5..."
5,MeshAI: Hospital Scheduling,meshai hospital scheduling our goal was to use...,"[0.6826685070991516, -0.6485515832901001, -1.8..."
6,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...,"[0.8958584666252136, -0.31829211115837097, -2...."
7,Queen’s University PPS: Android Waste Classifi...,queen s university pps android waste classific...,"[0.6723080277442932, -0.44362154603004456, -2...."
8,RBC: Topic Modelling,rbc topic modelling unfortunately specifics of...,"[0.7366456389427185, -0.4803701639175415, -1.7..."
9,TD Bank: Investing,td bank investing unfortunately specifics of t...,"[0.4428533613681793, -0.31850653886795044, -1...."


In [27]:
# Universal Sentence Encoder (USE)
# Add USE embedded vectors to df
df['use'] = np.array(embed(df['clean_text'])).tolist()

# Display
df

,projName,clean_text,proj2vec,use
0,Ace Interview Prep: Evaluating Confidence for ...,ace interview prep evaluating confidence for i...,"[0.6219965815544128, -0.5964304208755493, -1.9...","[0.01498220581561327, 0.042886197566986084, -0..."
1,Forest Ecosystem Solutions: Forest Ecosystem A...,forest ecosystem solutions forest ecosystem an...,"[0.7933398485183716, -0.5449119210243225, -1.9...","[-0.03158610314130783, 0.10768941789865494, -0..."
2,Kingston General Hospital: Forecasting Hospita...,kingston general hospital forecasting hospital...,"[0.7627487182617188, -0.6631572246551514, -1.9...","[0.02641439624130726, 0.10673921555280685, -0...."
3,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...,"[0.9004322290420532, -0.3092217445373535, -2.0...","[0.05598486214876175, -0.013048435561358929, -..."
4,LocateMotion: Diabetes Risk Prediction on Hete...,locatemotion diabetes risk prediction on heter...,"[0.9695004820823669, -0.7922118902206421, -1.5...","[-0.08370655030012131, 0.03755122423171997, 0...."
5,MeshAI: Hospital Scheduling,meshai hospital scheduling our goal was to use...,"[0.6826685070991516, -0.6485515832901001, -1.8...","[0.0329396016895771, 0.0900726467370987, 0.001..."
6,Loblaw Digital: Improving Product Categorizati...,loblaw digital improving product categorizatio...,"[0.8958584666252136, -0.31829211115837097, -2....","[0.05598486214876175, -0.013048435561358929, -..."
7,Queen’s University PPS: Android Waste Classifi...,queen s university pps android waste classific...,"[0.6723080277442932, -0.44362154603004456, -2....","[-0.026377025991678238, 0.10922525823116302, -..."
8,RBC: Topic Modelling,rbc topic modelling unfortunately specifics of...,"[0.7366456389427185, -0.4803701639175415, -1.7...","[0.006509436294436455, -0.013233556412160397, ..."
9,TD Bank: Investing,td bank investing unfortunately specifics of t...,"[0.4428533613681793, -0.31850653886795044, -1....","[-0.030831344425678253, -0.054855186492204666,..."


In [29]:
# Visualize Vectors
df['tsneP2V'] = hero.tsne(df['proj2vec'])
df['tsneUSE'] = hero.tsne(df['use'])

# Create scatter plot of doc2vec
hero.scatterplot(df, col='tsneP2V', title="Doc2Vec", hover_data=['projName'])

# Create scatter plot of uni. sent. enc.
hero.scatterplot(df, col='tsneUSE', title="U.S.E", hover_data=['projName'])